# Deep Learning: Introducción con TFLearn
## Sentiment analysis con TFLearn

Vamos a construir una Red Neuronal para aplicar Sentiment Analysis en review de peliculas. Vamos a utilizar [TFLearn](http://tflearn.org/), que es una librería de alto nivel contruida por encima de TensorFlow. TFLearn vuelve simple el contruir Redes Neuronales ya que solo se definen los layers. Se encarga de la mayoria de detalles por uno.


Comenzamos importando todos los módulos que vamos a utilzar, luego cargaremos la data y la prepararemos.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical

## Preparamos la data

Nuestro objetivo aquí es convertir los reviews en un vector de palabras. El vector de palabras tendrá elementos representantes del vocabulario total. Por ejemplo si la segunda posición del vector representa la palabra 'the', para cada review contaremos el número de veces que aparece 'the' y colocaremos ese valor en el vector. Les mostraré ejemplos conforme construimos el Input Data de las reviews.

### Leer u obtener la data

Usaremos la librería pandas para leer los reviews y sus etiquetas (labels) positiva/negativa de un archivo coma separado.  Los datos que estamos utilizando ya han sido preprocesados un poco y sabemos que sólo utiliza caracteres en minúsculas. Si tubieras datos sin pre procesar aquí abría que añadir un paso de limpiar los datos. Con ello solucionamos diferentes variaciones de una misma palabra por ejemplo 'the', 'The', 'tHe' y 'THE'.

In [ ]:
reviews = pd.read_csv('reviews.txt', header=None)
labels = pd.read_csv('labels.txt', header=None)

### Conteo de la frecuencia de las  palabras

Para comenzar debemos contar la frecuencia con que cada palabra aparece en los datos. Usaremos este recuento para crear un vocabulario que usaremos para codificar las reviews. A ese resultado se le llama Saco de Palabras ([bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model)). Lo utilizaremos para seleccionar el vocabulario y construir nuestro vector de palabras. Trataremos de implementarlo utilizando un [Counter class](https://docs.python.org/2/library/collections.html#collections.Counter).

> **Ejercicio:** Crear un bag of words de las reviews y asignarlo a `total_counts`. Las reviews se almacenan en el `reviews` [Pandas DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). Si deseas las reviews como un array de Numpy, utiliza `reviews.values`. Puedes iterar las filas en el DataFrame con `for idx, row in reviews.iterrows():` ([documentación](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.iterrows.html)). Cuando dividas las reviews en palabras, usa `.split(' ')` y no `.split()` para que nos salga el mismo resultado.

In [ ]:
from collections import Counter

total_counts = # bag of words here

print("Total words in data set: ", len(total_counts))

Mantengamos las 10000 primeras palabras más frecuentes. La mayoría de las palabras del vocabulario se usan raramente por que tendrán poco efecto en nuestras predicciones. A continuación, ordenaremos `vocab` por conteo y solo usaremos las 10000 más frecuentes.

In [ ]:
vocab = sorted(total_counts, key=total_counts.get, reverse=True)[:10000]
print(vocab[:60])

¿Cuál es la última palabra en nuestro vocabulario? Podemos usar esto para juzgar si 10000 es demasiado pequeño. Si la última palabra es bastante común, es probable que tengamos que guardar más palabras.

In [ ]:
print(vocab[-1], ': ', total_counts[vocab[-1]])

La última palabra en nuestro vocabulario se muestra en 30 reviews de 25000. Creo que es justo decir que esto es una pequeña proporción de comentarios. Probablemente estamos bien con este número de palabras.

**Nota:** Cuando lo ejecutes, puede que veas una palabra diferente, pero sigue siendo de valor `30`. Eso es porque hay varias palabras con la misma cantidad de ocurrencias, y la `Counter` class no garantiza cual va a retornar en ese caso.

Ahora para cada review, crearemos un vector de palabras. Primero necesitamos hacer un mapeo de palabras a un índice, bastante fácil de hacer con una comprensión del diccionario.

> **Ejercicio:** Crear un dictionario llamado `word2idx` que asigne cada palabra del vocabulario un índice. La primera palabra en `vocab` tiene el índice `0`, la segunda palabra tiene índice `1`, y así sucesivamente.

In [ ]:
word2idx = ## create the word-to-index dictionary here

### Texto a vector, función

Ahora podemos escribir una función que convierta un texto a un vector de palabras. La función recibe un string de palabras y retorna un vector con el conteo de palabras. Un algoritmo genérico para hacer eso:

* Inicializa el vector de palabras con [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html), debería ser del largo del vocabulario.
* Separa el input string del texto en una lista de palabras con `.split(' ')`.
* Por cada palbra en esa lista, incrementa el elemento en el índice asociado a esa palabra, que lo obtienes de `word2idx`.

**Nota:** Como no todas las palabras que apareazcan están en el diccionario `vocab`, tendrás un error de key "index out of bounds", si se trata de acceder a una palabra que no esté. Puedes usar el método `.get` del diccionario `word2idx` para especificar el valor default de retorno cuando de un error. Por ejemplo, `word2idx.get(word, None)` retorna `None` si `word` no existe en el diccionario.

In [ ]:
def text_to_vector(text):
    
    pass

Si lo hiciste bien el siguiente código retornará:

```
text_to_vector('The tea is for a party to celebrate '
               'the movie so she has no time for a cake')[:65]
                   
array([0, 1, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])
```       

In [ ]:
text_to_vector('The tea is for a party to celebrate '
               'the movie so she has no time for a cake')[:65]

Ahora, itera en todas las reviews y conviertelas en un vector de palabras.

In [ ]:
word_vectors = np.zeros((len(reviews), len(vocab)), dtype=np.int_)
for ii, (_, text) in enumerate(reviews.iterrows()):
    word_vectors[ii] = text_to_vector(text[0])

In [ ]:
# Printing out the first 5 word vectors
word_vectors[:5, :23]

### Train, Validation, Test sets

Ahora que tenemos los vectores de palabras, estamos listos para dividir nuestros datos en train, validation y test sets. Utilizaremos para entrenar el train set, setearemos los hyperparameters con el validation set, y hasta el final evaluaremos el performace de la red neuronal con el test set. 
Aquí estamos usando la función `to_categorical` de TFLearn para remodelar los datos objetivo para que tengamos 2 unidades de output, para podamos clasificar utilizando una función de activación softmax. Nosotros no crearemos el set de validación eso lo hara TFLearn por nosotros.

In [ ]:
Y = (labels=='positive').astype(np.int_)
records = len(labels)

shuffle = np.arange(records)
np.random.shuffle(shuffle)
test_fraction = 0.9

train_split, test_split = shuffle[:int(records*test_fraction)], shuffle[int(records*test_fraction):]
trainX, trainY = word_vectors[train_split,:], to_categorical(Y.values[train_split], 2)
testX, testY = word_vectors[test_split,:], to_categorical(Y.values[test_split], 2)

In [ ]:
trainY

## Construyendo la Red Neuronal

[TFLearn](http://tflearn.org/) te permite construir la red [definiendo los layers](http://tflearn.org/layers/core/). 

### Input layer

Para el input layer, solo necesitas indicar cuantas neuronas tendrá. Por ejemplo, 

```
net = tflearn.input_data([None, 100])
```
esto creara una red con 100 neuronas de input. El primer elemento de la lista, `None` en este caso, setea el tamaño del batch. Colocar `None` ahí indica que utilize el valor por default.

El número de inputs de la red debe ser del mimso tamaño de la data. Por ejemplo, si nuestro vector de palabras tiene 5000 elementos, entonces colocamos 5000 neuronas de input.


### Añadir layers

Para agregar hidden layers, se utiliza

```
net = tflearn.fully_connected(net, n_units, activation='ReLU')
```

Esto agrega una layer que es fully connected donde cada una de las neuronas de el layer anterior está conectadas a esta layer. El primer argumento `net` es la Red que creaste con `tflearn.input_data`. Le indica a la Red que utilice el output de la layer anterior como input de la siugiente layer.
Tu puedes especificar el número de neuronas en el layer con `n_units`, y setea el activation function con el keyword `activation`. Tu puedes agregar varias layers a la red utilizando a varias veces `net = tflearn.fully_connected(net, n_units)`.

### Output layer

La última layer que se agrega es el output layer. Por lo que, el número de neuronas tiene que ser igual al target data. En este caso estamos clasificando entre 2 clases, sentimiento positivo y negativo. También el activation function que depende del modelo.
Estamos tratando de clasificar si el input (el review) pertenece a una de las dos clases, por lo que utilizaremos softmax.

```
net = tflearn.fully_connected(net, 2, activation='softmax')
```

### Training

Para indicar como va a entrenar la Red, usamos

```
net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')
```

Otra vez, esto sucede en la Red que estas construyendo. Los keywords:
* `optimizer` setea el método de entrenamiento, utiliza stochastic gradient descent
* `learning_rate` es el learning rate
* `loss` determina como la red calcula el error. En este ejemplo, categorical cross-entropy.

Finalemente colocas todo junto creado un modelo con `tflearn.DNN(net)`. Así que termina viendose algo como:
```
net = tflearn.input_data([None, 10])                          # Input
net = tflearn.fully_connected(net, 5, activation='ReLU')      # Hidden
net = tflearn.fully_connected(net, 2, activation='softmax')   # Output
net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')
model = tflearn.DNN(net)
```

> **Ejercicio:** Abajo en la función `build_model()`, colocas todo junto en la Red usando TFLearn. Tu escoges cuantas layers utilizar, y cuantas neuronas tendrán las hidden layers.

In [ ]:
# Network building
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    #### Your code ####
    
    model = tflearn.DNN(net)
    return model

## Inicializar el modelo

Lo siguiente es que llamamos la función `build_model()` para construir el modelo. No agregué parametros pero si lo deseas puedes ver en el API de TFLearn.

> **Note:** Es posible que te salgan varios warnings. TFLearn usa muchas cosas deprecated en TensorFlow. Pronto estará actualizado.

In [ ]:
model = build_model()

## Entrenar la Red Neuronal

Ahora que ya construimos la Red, que la refiere la variable `model`, ahora podemos ajustarlo a los datos. Pare eso usamos el método `model.fit`. Colocas en los training features `trainX` y en los training targets `trainY`. Abajo coloqué `validation_set=0.1` el cual reserva el 10% de la data para el validation set. También puedes setear el el tamaño del batch y el número de epochs con los keywords `batch_size` y `n_epoch`. Abajo está el código para para ajustar la red con nuestro vector de palabras (fit).

Puedes volver a ejecutar `model.fit` para entrenar la red más, si crees que se puede aumentar el validation accuracy. Recuerda, los ajustes de hyperparameter se hace con el validation set. **Solo usa el testing set hasta que ya estas completamente seguro dde que la Red está lista**

In [ ]:
# Training
model.fit(trainX, trainY, validation_set=0.1, show_metric=True, batch_size=128, n_epoch=10)

## Testing

Después de que creas que los hyperparameters esten listos, puedes probar la Red con el test set para medir su rendimiento, que tan bien clasifica. Recuerda, *solo haz eso hasta que hallas terminado de probar los hyperparameters*.

In [ ]:
predictions = (np.array(model.predict(testX))[:,0] >= 0.5).astype(np.int_)
test_accuracy = np.mean(predictions == testY[:,0], axis=0)
print("Test accuracy: ", test_accuracy)

## Ahora probémoslo!
#Crea tu propia review de pelicula.
PD: el training data está en inglés, por lo que nuestro vector de palabras está en inglés. Entonces el review lo debemos escribir en inglés.

In [ ]:
# Helper function that uses your model to predict sentiment
def test_sentence(sentence):
    positive_prob = model.predict([text_to_vector(sentence.lower())])[0][1]
    print('Sentence: {}'.format(sentence))
    print('P(positive) = {:.3f} :'.format(positive_prob), 
          'Positive' if positive_prob > 0.5 else 'Negative')

In [ ]:
sentence = "Star Trek Beyond is the best movie of 2016, too epic for my bones."
test_sentence(sentence)

sentence = "It's amazing anyone could be talented enough to make something this spectacularly awful"
test_sentence(sentence)